<a href="https://colab.research.google.com/github/rohitsurya26/WiDS-/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment_Analysis
#### By Rohit Suryavanshi
Work Timeline doc: https://docs.google.com/document/d/1IXqmf_iipkzNDwZqWczPdeCnnNbQvESU1ktOREng1D0/edit?usp=sharing

In [ ]:
import numpy as np
import pandas as pd
import math
import nltk
nltk.download("popular")
#nltk.download()

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
url = 'https://raw.githubusercontent.com/rohitsurya26/WiDS-/main/training.csv?token=GHSAT0AAAAAABQZ6QUIRAC5LYR25NLDXITWYPODPZQ'
df = pd.read_csv(url)

### Labels:

0.   sadness
1.   joy
2.   love
3.   anger
4.   fear
5.   surprise

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize

# Pre-Processing

## Remove Punctuations
#### Ref: https://studymachinelearning.com/text-preprocessing-removal-of-punctuations/

In [ ]:
import string
regular_punct = list(string.punctuation)
def remove_punctuation(text):
    punct_list = regular_punct
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()
df['punc_text']=df['text'].apply(remove_punctuation)

## Tokenising Text

In [ ]:
#nltk.download('punkt')
df['tokenised']=df.apply(lambda row: nltk.word_tokenize(row['punc_text']), axis=1)

## Lemmatization

In [ ]:
from nltk.stem import PorterStemmer
porter_stemmer=PorterStemmer()
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
text=df['tokenised']
text=text.tolist()
for i in range(len(text)):
    text[i]=[lemmatizer.lemmatize(word=word,pos='v') for word in text[i]]
df['lemmatized']= pd.Series((i for i in text))

## Removal of Stopwords
##### Ref: https://stackabuse.com/removing-stop-words-from-strings-in-python/#usingpythonsnltklibrary¶
##### Ref: https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe

In [ ]:
#nltk.download('stopwords')
stop = stopwords.words('english')
df['string_text']= df['lemmatized'].str.join(" ")
df['cleaned_text'] = df['string_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

### New Dataset with Pre-Processed Text

In [ ]:
new_df = pd.DataFrame(df['cleaned_text'])
new_df['label']=df['label']
new_df.to_csv('pre-processed_data.csv',index=False)

## Implementation

In [ ]:
## Splitting into training and test(C.V.) data set
n=len(new_df['label'])
idx_train = np.random.choice(n,int(n),replace=False) ## Gives us a list
#idx_test = np.random.choice(n,int(n*0.2),replace=False) ## int bcuz n*0.2 is float

In [ ]:
df_train=new_df.loc[idx_train]
x_train=df_train['cleaned_text']
y_train=df_train['label']

In [ ]:
# df_train,df_test=new_df.loc[idx_train],new_df.loc[idx_test]
# x_train,x_test=df_train['cleaned_text'],df_test['cleaned_text']
# y_train,y_test=df_train['label'],df_test['label']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

corpus_train=x_train.tolist()

vectorizer =  CountVectorizer() 
#vectorizer =  TfidfVectorizer()

X_train = vectorizer.fit_transform(corpus_train)

vocab=vectorizer.get_feature_names_out()
vocab_size=len(vocab)

In [ ]:
def tagging(corpus,vocab):
    keys=list(range(len(vocab)))
    keys=[x+1 for x in keys]
    text_dict = dict(zip(vocab,keys))
    n=len(corpus)
    lst=[1]*n
    for i in range(n):
        string=corpus[i].split()
        lst[i]=[text_dict.get(j) for j in string]
    return lst
def padding(x,n_max):
    new_x=x
    n=len(x)
    for i in range(n):
        new_x[i] = x[i] + [0] * (n_max - len(x[i]))
    return new_x    

n_max=40 ## for corpus_train    

x_encoded=tagging(corpus_train,vocab)   
x_padded=padding(x_encoded,n_max)

x_train=np.array(x_padded,dtype='float32')
idx_NaNs = np.isnan(x_train)
x_train[idx_NaNs]=0
# lst_nan=np.where(np.isnan(x_train).any(axis=1)) # rows where any value is nan
# x_train=np.delete(x_train,lst_nan,axis=0)
# y_train=np.delete(y_train,lst_nan,axis=0)


In [ ]:
a = np.array([[1, 2, 3], [0, 3, float('nan')]])
where_are_NaNs = np.isnan(a)
a[where_are_NaNs] = 0
print(a)

[[1. 2. 3.]
 [0. 3. 0.]]


In [ ]:
url='https://raw.githubusercontent.com/rohitsurya26/WiDS-/main/test.csv?token=GHSAT0AAAAAABQZ6QUJ4FG7Y2PHQUCDDDYOYPODQJA'
df_test=pd.read_csv(url)

In [ ]:
## REMOVE PUNCTUATION
df_test['text']=df_test['text'].apply(remove_punctuation)
## TOKENISATION
df_test.text=df_test.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
## LEMMATIZATION
submit=df_test['text']
submit=submit.tolist()
for i in range(len(submit)):
    submit[i]=[lemmatizer.lemmatize(word=word,pos='v') for word in submit[i]]
## REMOVE STOPWORDS
submit=pd.Series(submit)
df_test['string_text']= submit.str.join(" ")
df_test['cleaned_text'] = df_test['string_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
submit=df_test['cleaned_text']

In [ ]:
corpus_submit=submit.tolist()

In [ ]:
submit_encoded=tagging(corpus_submit,vocab)
submit_padded=padding(submit_encoded,n_max)    
x_test=np.array(submit_padded)
x_test=np.array(x_test,dtype='float32')
idx_NaNs = np.isnan(x_test)
x_test[idx_NaNs]=0

# Deep Learning
### Used Tensorflow and Keras

In [ ]:
import tensorflow 
from tensorflow import keras

### Used One-hot Encoding and Padding to input it into Embedding Layer

In [ ]:
# from tensorflow.keras.preprocessing.text import one_hot
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# encoded=[one_hot(i,vocab_size) for i in corpus_train]
# padded=pad_sequences(encoded,maxlen=max_length,padding='post')

In [ ]:
# x_train=np.array(x_train,dtype='float32')

In [ ]:
# lst_nan=np.where(np.isnan(x_train).any(axis=1)) # rows where any value is nan
# x_train=np.delete(x_train,lst_nan,axis=0)
# y_train=np.delete(y_train,lst_nan,axis=0)

### Developing Model and Fitting

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D

embed_size=45
model = Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=embed_size,mask_zero=True,input_length=n_max))
# model.add(Flatten())
model.add(Conv1D(32,15,activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=64,activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_39 (Embedding)    (None, 40, 45)            544860    
                                                                 
 conv1d_28 (Conv1D)          (None, 26, 32)            21632     
                                                                 
 global_average_pooling1d_34  (None, 32)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_67 (Dense)            (None, 64)                2112      
                                                                 
 dense_68 (Dense)            (None, 6)                 390       
                                                                 
Total params: 568,994
Trainable params: 568,994
Non-trainable params: 0
_______________________________________________

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=1,shuffle=True)

Epoch 1/5
16000/16000 [==============================] - 136s 8ms/step - loss: 0.9107 - accuracy: 0.6094
Epoch 2/5
16000/16000 [==============================] - 134s 8ms/step - loss: 0.3347 - accuracy: 0.8836
Epoch 3/5
16000/16000 [==============================] - 133s 8ms/step - loss: 0.1945 - accuracy: 0.9316
Epoch 4/5
16000/16000 [==============================] - 133s 8ms/step - loss: 0.1263 - accuracy: 0.9546
Epoch 5/5
16000/16000 [==============================] - 130s 8ms/step - loss: 0.0851 - accuracy: 0.9693


In [ ]:
loss,accuracy=model.evaluate(x_train,y_train)
print('Loss:',loss)
print('Accuracy:',accuracy)

500/500 [==============================] - 2s 3ms/step - loss: 0.0431 - accuracy: 0.9845
Loss: 0.04306742548942566
Accuracy: 0.984499990940094


### Prediction

In [ ]:
p=model.predict(x_test)
b = np.zeros_like(p)
b[np.arange(len(p)), p.argmax(1)] = 1
kaggle=b
def labelling(matrix):
    n=len(matrix)
    lst=[1]*n
    for i in range(n):
        arr=matrix[i]
        arr=list(arr)
        idx=arr.index(max(arr))
        lst[i]=idx        
    return lst
a=labelling(kaggle)        

### CSV Submission File

In [ ]:
kaggle=pd.DataFrame(columns=['id','label'])
n=len(a)
k=np.arange(n)+1
kaggle['id']=pd.Series(k)
kaggle['label']=pd.Series(a)
# print(len(kaggle['id']))
kaggle.to_csv('submission_colab.csv',index=False)

# Naive Bayes Implementation

## Basic Way to Approach
1. Splitting dataframe in 80-20 way for training and test(C.V.) datasets
2. Import TF-IDF vectorizer and corpus from x_train and x_test
3. Apply vectorizer on corpus and obtain array containing TF-IDF values for each word
4. Apply Complement Naive Bayes

In [ ]:
n=len(new_df['label'])
idx_train = np.random.choice(n,int(n),replace=False)

In [ ]:
df_train=new_df.loc[idx_train]
x_train=df_train['cleaned_text']
y_train=df_train['label']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

corpus_train=x_train.tolist()

vectorizer =  CountVectorizer() 
#vectorizer =  TfidfVectorizer()

X_train = vectorizer.fit_transform(corpus_train)

#### Implemented Scaling - Decreased Accuracy

In [ ]:
from sklearn import preprocessing
# scaler1 = preprocessing.StandardScaler(with_mean=False).fit(X_train) ## in [-2,2]
# scaler2 = preprocessing.StandardScaler(with_mean=False).fit(X_test) ## with_mean=False is used for sparse matrices
# X_train_scaled = scaler1.transform(X_train)
# X_test_scaled = scaler2.transform(X_test)

# min_max_scaler = preprocessing.MinMaxScaler() # to fit array in [0,1] range

# min_max_scaler = preprocessing.MaxAbsScaler()
# # This estimator scales and translates each feature individually such that the maximal absolute value of each feature 
# # in the training set will be 1.0. 
# # It does not shift/center the data, and thus does not destroy any sparsity.
# # This scaler can also be applied to sparse CSR or CSC matrices.

# X_train_scaled = min_max_scaler.fit_transform(X_train)
# X_test_scaled = min_max_scaler.fit_transform(X_test)

# X_train_scaled.array
# X_test_scaled.toarray()
# y_train.array
# y_test.array

X_train_scaled=X_train
#X_test_scaled=X_test

In [ ]:
from sklearn.naive_bayes import BernoulliNB ## something new score - 0.7
from sklearn.naive_bayes import CategoricalNB ##sparse error 
from sklearn.naive_bayes import GaussianNB ##sparse error
from sklearn.naive_bayes import ComplementNB ## dimension mismatch - max_score - 0.933125
from sklearn.naive_bayes import MultinomialNB ## dimension mismatch - 0.8

model = ComplementNB() ## 0.7240

### ENSEMBLE LEARNING
##### Decreased Accuracy since Complement NB was way stronger than multi and Bernoulli

In [ ]:
### ENSEMBLE LEARNING

# model2 = MultinomialNB() ## 0.61
# model3 = BernoulliNB() ## 0.61400
# Cnb=model1.fit(X_train_scaled, y_train)
# Mnb=model2.fit(X_train_scaled, y_train)
# Bnb=model3.fit(X_train_scaled, y_train)

# from sklearn.ensemble import VotingClassifier
# #create a dictionary of our models
# estimators=[('Complement', Cnb), ('Multinomial', Mnb), ('Bernoulli', Bnb)]
# #create our voting classifier, inputting our models
# ensemble = VotingClassifier(estimators, voting='hard')
# submission=ensemble.fit(X_train_scaled, y_train)
# ensemble.score(X_test_scaled, y_test)

In [ ]:
submission=model.fit(X_train_scaled, y_train) 

In [ ]:
df_test=pd.read_csv('test.csv')
## REMOVE PUNCTUATION
df_test['text']=df_test['text'].apply(remove_punctuation)
## TOKENISATION
df_test.text=df_test.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
## LEMMATIZATION
submit=df_test['text']
submit=submit.tolist()
for i in range(len(submit)):
    submit[i]=[lemmatizer.lemmatize(word=word,pos='v') for word in submit[i]]
## REMOVE STOPWORDS
submit=pd.Series(submit)
df_test['string_text']= submit.str.join(" ")
df_test['cleaned_text'] = df_test['string_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
submit=df_test['cleaned_text']


In [ ]:
n=len(submit)
corpus_submit=submit.tolist()
submit = vectorizer.transform(corpus_submit)
#submit_scaled = min_max_scaler.fit_transform(submit)
submit=submission.predict(submit)
kaggle=pd.DataFrame(columns=['id','label'])
k=np.arange(n)+1
kaggle['id']=pd.Series(k)
kaggle['label']=pd.Series(submit)
len(kaggle['id'])
kaggle.to_csv('submission_corrected.csv',index=False)

## WITHOUT SCALING - 0.73400, IMPROVES BY 0.1
## WITH SCALING, DOWN BY 0.01 FOR MAX ABS 
##               DOWN BY 0.12 FOR STANDARD SCALER  
## WITHOUT STOPWORDS - 0.654
## USING COUNT_VECTORIZER - 0.674
## with full dataset as train - 0.728 and score - 0.93
## BY REPETITION, 0.74200
## Spell checker and contractions brings down score
## CURRENT - FULL SET AND ADDED PUNCTUATIONS - increase by 0.004

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers

In [ ]:
x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])
print(x)
print(x.shape)
print(x.dtype)
x @ tf.transpose(x)

In [ ]:
tdf=pd.read_csv('play_data.csv')
y=tdf.label.values
x=np.column_stack((tdf.YearsExperience.values,tdf.Salary.values))
np.random.shuffle(x)  
#type(x)

In [ ]:
#np.random.choice()
model=keras.Sequential([keras.layers.Dense(16,input_shape=(2,),activation='relu'),keras.layers.Dense(2,activation='relu')])
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(x,y,batch_size=16)

In [ ]:
x=X_train.toarray()
y=y_train.values
print(x.shape)

In [ ]:
model=keras.Sequential([keras.layers.Dense(16,input_shape=(12107,),activation='relu'),keras.layers.Dense(6,activation='relu')])
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
y=new_df.label.values
model.fit(x[0:1000],y[0:1000],batch_size=1)